In [1]:
# Standard Data Science Helpers
import numpy as np
import pandas as pd
import scipy

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

import cufflinks as cf
cf.set_config_file(world_readable=True, theme="pearl")
cf.go_offline(connected=True)

# Extra options
pd.options.display.max_rows = 10
pd.options.display.max_columns = 25
# Show all code cells outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


In [2]:
import pickle

with open('out/test_summaries.pkl', 'rb') as fin:
    tests = pickle.loads(fin.read())
    
len(tests)

2967

In [3]:
train_meta = pd.read_csv('out/train_meta.csv', index_col='process_id')
test_meta = pd.read_csv('out/test_meta.csv',index_col='process_id')

In [4]:
train_meta.head()

,pre_rinse,caustic,intermediate_rinse,acid,final_rinse,sequence,phases
process_id,,,,,,,
20001,1,1,1,1,1,11111,['pre_rinse' 'caustic' 'intermediate_rinse' 'a...
20002,1,1,0,0,1,11001,['pre_rinse' 'caustic']
20003,1,1,1,1,1,11111,['pre_rinse' 'caustic' 'intermediate_rinse' 'a...
20004,1,1,1,1,1,11111,['pre_rinse' 'caustic' 'intermediate_rinse' 'a...
20005,1,0,0,1,1,10011,['acid']


In [5]:
test_meta.head()

,pre_rinse,caustic,intermediate_rinse,acid,final_rinse,sequence,phases,summary_length
process_id,,,,,,,,
20000,1,1,1,1,1,11111,['pre_rinse' 'caustic'],320
20006,1,1,0,0,1,11001,['pre_rinse' 'caustic'],320
20007,1,1,1,1,1,11111,['pre_rinse' 'caustic' 'intermediate_rinse'],480
20009,1,1,1,1,1,11111,['pre_rinse' 'caustic' 'intermediate_rinse'],480
20010,1,1,1,1,1,11111,['pre_rinse' 'caustic' 'intermediate_rinse' 'a...,480
